In [40]:
import torch
from torch import nn
from torch.nn import functional as F
from torchinfo import summary



In [41]:
#假设图片的尺寸
data = torch.ones(size=(20,3,227,227))

In [42]:
#建立模型
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,96,kernel_size=11,stride=4)#输入通道是3，输出通道是96，卷积核（过滤器）大小是11*11，步行是4
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)#卷积核是3*3，步行等于2，产生一个重复重叠感受野

        self.conv2 = nn.Conv2d(96,256,kernel_size=5,padding=2)#输入通道是96，输出通道是256，卷积核（过滤器）大小是5*5，填充是2
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2)#步行等于2，产生一个重复重叠感受野

        self.conv3 = nn.Conv2d(256,384,kernel_size=3,padding=1)#输入通道是256，输出通道是384，卷积核（过滤器）大小是3*3，填充是1
        self.conv4 = nn.Conv2d(384,384,kernel_size=3,padding=1)#输入通道是384，输出通道是384，卷积核（过滤器）大小是3*3，填充是1
        self.conv5 = nn.Conv2d(384,256,kernel_size=3,padding=1)#输入通道是384，输出通道是256，卷积核（过滤器）大小是3*3，填充是1

        self.pool3 = nn.MaxPool2d(kernel_size=3,stride=2)#步行等于2，产生一个重复重叠感受野

        #FC
        self.fc1 = nn.Linear(6*6*256,4096)#将展平后的特征输入到第一个全连接层，输入维度为 6*6*256，输出 4096 个神经元
        self.fc2 = nn.Linear(4096,4096)#增加训练的强度，增强非线性表达能力
        self.fc3 = nn.Linear(4096,1000)#它将 4096 个神经元的输出映射到 1000 个类别。这里 1000 是因为原始的 AlexNet 是用来做 ImageNet 分类任务的，ImageNet 数据集有 1000 个类别

#写激活函数，先经过卷积层1，再经过一次池化层1，然后再一个卷积层2，池化层2，再走3个卷积层345，然后再走一个池化层3，
def forward(self,x):
    x = F.relu(self.conv1(x))#使用relu函数
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = F.relu(self.conv5(x))
    x = self.pool3(x)
    x = x.view(-1,6*6*256)#将从池化层3 输出的尺寸为6*6，256通道的尺寸拉平成一维的一个向量，因为全连接层必须是一维的向量。
    x = F.relu(F.dropout(self.fc1(x),0.5))#使用dropout函数防止过拟化，使用relu函数引入非线性
    x = F.relu(F.dropout(self.fc2(x),0.5))#使用dropout函数防止过拟化，使用relu函数引入非线性
    output = F.softmax(self.fc3(x),dim=1)#使用 Softmax 函数将这些分数转化为概率分布。




In [43]:
net = Model()

In [44]:
net = (data)